In [19]:
# !pip install datasets==1.18.3 python-dotenv==0.19.2
# !pip install tokenizers==0.19
# !pip install accelerate

In [1]:
user = !echo $USER
user = user.get_list()[0]
print(user)

kmallick


In [2]:
import os
os.environ['HF_HOME'] = f'/scratch/gilbreth/{user}/'
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [3]:
import gc

gc.collect()

168

In [4]:
!export HF_HOME=/scratch/gilbreth/$USER/

In [5]:
BASE_PATH = f'/scratch/gilbreth/{user}/'

In [6]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

from util import nethook
from util.generate import generate_interactive, generate_fast

from experiments.py.demo import demo_model_editing, stop_execution

In [7]:
MODEL_NAME = "gpt2-xl"

In [8]:
model, tok = (
    AutoModelForCausalLM.from_pretrained(MODEL_NAME, low_cpu_mem_usage=False, cache_dir=BASE_PATH).to(
        "cuda"
    ),
    AutoTokenizer.from_pretrained(MODEL_NAME, cache_dir=BASE_PATH),
)
tok.pad_token = tok.eos_token
model.config

2024-11-15 11:33:01.902394: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731688382.006986   70991 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731688382.056385   70991 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-15 11:33:02.253286: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
with open('pfw.txt', 'r') as f:
    data = f.read()

In [28]:
# Adjust chunk_size based on model's max input size
# Adjust stride based on desired overlap
def chunkify(text, chunk_size=1024, stride=256):
    chunks = []
    for i in range(0, len(text), stride):
        chunk = text[i:i + chunk_size]
        if len(chunk) == chunk_size:
            chunks.append(chunk)
    return chunks

text_chunks = chunkify(data)

In [29]:
encoded_chunks = tok(text_chunks, truncation=True, padding=True, return_tensors="pt")

In [30]:
from torch.utils.data import Dataset

class TextDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = item['input_ids'].clone()
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

train_dataset = TextDataset(encoded_chunks)

In [31]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir=os.path.join(BASE_PATH, "results"),
    num_train_epochs=50,
    per_device_train_batch_size=4,
    warmup_steps=750,
    learning_rate=3e-5,
    weight_decay=0.01,
    logging_dir=os.path.join(BASE_PATH, "logs"),
    logging_steps=50,
    save_steps=500,                   # Save checkpoint every 500 steps
    save_total_limit=1  
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [32]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [33]:
torch.cuda.empty_cache()

In [34]:
trainer.train()

Step,Training Loss
50,5.029300
100,0.902500
150,0.155100
200,0.033500
250,0.023700
300,0.015000
350,0.005700


TrainOutput(global_step=350, training_loss=0.8806908971071243, metrics={'train_runtime': 461.9107, 'train_samples_per_second': 3.031, 'train_steps_per_second': 0.758, 'total_flos': 4871123953920000.0, 'train_loss': 0.8806908971071243, 'epoch': 50.0})

In [35]:
trainer.save_model(os.path.join(BASE_PATH, "fine_tuned_model/"))

In [36]:
tok.save_pretrained(os.path.join(BASE_PATH, "fine_tuned_model/"))

('/scratch/gilbreth/dparveez/fine_tuned_model/tokenizer_config.json',
 '/scratch/gilbreth/dparveez/fine_tuned_model/special_tokens_map.json',
 '/scratch/gilbreth/dparveez/fine_tuned_model/vocab.json',
 '/scratch/gilbreth/dparveez/fine_tuned_model/merges.txt',
 '/scratch/gilbreth/dparveez/fine_tuned_model/added_tokens.json',
 '/scratch/gilbreth/dparveez/fine_tuned_model/tokenizer.json')